<a href="https://colab.research.google.com/github/hululuzhu/gpt-j/blob/main/GPT_J_6B_Iference_demo_and_chinese_coding_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Premier Colab GPU instance required, cannot fix TPU for free class :(

In [8]:
# Check GPU
!pip install -q nvidia-ml-py3
import nvidia_smi
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
# card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
nvidia_smi.nvmlShutdown()

print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

assert info.free > 4e10, (
    "Looks like your GPU is busy or not having enough memory to host GPT-J-6B")

Total memory: 42949672960
Free memory: 42481418240
Used memory: 468254720


In [11]:
# Check main memory
import psutil
vm_info = psutil.virtual_memory()
assert vm_info.available > 4e10, (
    "Looks like your main memory is not enough to initialize GPT-J-6B")

## Load Model, move to GPU and start inference

In [6]:
!pip install -q transformers
from transformers import GPTJForCausalLM, AutoTokenizer
import torch

model = GPTJForCausalLM.from_pretrained("EleutherAI/gpt-j-6B",
                                        torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.2 MB/s eta 0:00:00


In [17]:
# Push model to GPU
model = model.to('cuda')

In [30]:
def inference(prompt, max_tokens=128, top_p=0.95, temperature=0.8):
  """See decoding params at See https://huggingface.co/blog/how-to-generate."""
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids
  input_ids = input_ids.to('cuda')
  gen_tokens = model.generate(
      input_ids,
      do_sample=True,
      top_p=top_p,
      temperature=0.9,
      max_length=max_tokens,
      pad_token_id=tokenizer.eos_token_id
  )
  gen_text = tokenizer.batch_decode(gen_tokens)[0]
  print(gen_text)

In [33]:
inference("""question: how to write a python program to get the largest of 3 integer numbers?
answer:""")

question: how to write a python program to get the largest of 3 integer numbers?
answer: use max() function.
how to get a sum of 3 numbers, like 3+3+3 is equal to 9

A:

To get the largest of 3 numbers, you can use this:
>>> max(3, 7, 12)
12

You could do something similar with a sum of 3 numbers. The code would look something like this:
>>> sum(3, 3, 3)
9

A:

For the first question, you can use max(), like this


In [34]:
inference("""问题：如何能用python计算三个整数中最大的那个数？
答案:""")

问题：如何能用python计算三个整数中最大的那个数？
答案:
from math import max, max(a, b, c)
print max(max(a, b), c)

递归方式：
def maxnum(a, b, c):
    if a > b:
        return maxnum(a, c, b)


In [35]:
inference("""question: which programming language is the best?
answer:""")

question: which programming language is the best?
answer: python
question: you do a lot of programming in python, why python and not perl or php?
answer: python has the best python community, python has good libraries, python has the best text processing libraries, and python is more fun than perl

Question: why do you think python is the best?
answer: for many reasons

Question: which programming language is best for web development?
answer: PHP
question: which programming language is the best for enterprise applications?
answer: Java

Question: why do you think perl or php is


In [36]:
inference("""问题：哪个编程语言是最好的编程语言？
答案:""")

问题：哪个编程语言是最好的编程语言？
答案:

C# 优秀编程语言，
C# 语法非常好
C# 语法灵活
C# 文档是全球的
C# 的开�


In [37]:
inference("""问题：php是世界上最好的编程语言吗？
答案:""")

问题：php是世界上最好的编程语言吗？
答案: 按照统计数据，目前支持百分之三半的框架，与像Python, C#, Java这样的定义，编译语言支


In [42]:
inference("""问题：写一个java程序从3个输入数字中获取第二大整数
答案:""", 256)

问题：写一个java程序从3个输入数字中获取第二大整数
答案: 1、找出所有三个数的和，找出第二个数；2、使用列表分别添加每一个三个数为单个元素，
再找出第二大数。（题目里有教程）

```
class Solution {
    public int Solution(int[] nums) {
        int[] nums1=new int[3];
        int[] nums2=new int[3];
        int[] nums3=new int[3];
   


In [41]:
inference( """问题：写一个shell程序从输入txt文件中读取最长的string
答案:""", 512)

问题：写一个shell程序从输入txt文件中读取最长的string
答案:不太有意义，仅仅用于编程
说明:从文件中读取的string必须有可能只有最长一个
而且，还是读取第一个的,而不是最后一个
另外，string有几种不同的序列化形式

统计类
用进行字符数统计的方法有很多,请放心阅读诸如:

http://www.gnu.org/software/gsl/manual/html_node/Statistical-Functions.html
http://www.cs.pitt.edu/~greg/cs253/cs253/handout/algorithms.pdf
http://en.wikipedia.org/wiki/Algorithms_for_Efficient_Computing_of_Statistical_Functions

讨论：

- 采用gsl库的函数统计类
- 关闭多倍的基准测试
- 自由库的自定义基准测试

如何统计字符数

统计类的方法和关键点

字符数统计方法

一直用的有统计算法：

基准：




In [43]:
inference("""问题：写一个css实现渐变紫红色圆角
答案:""", 512)

问题：写一个css实现渐变紫红色圆角
答案:提供一个css class：.trans-25,阻止渐变的外部扩展,不会影响内部其他功能。
```
/* 轮播*/
.trans-25 {
	-webkit-transform: translate(-25%, -25%);
	-moz-transform: translate(-25%, -25%);
	-ms-transform: translate(-25%, -25%);
	-o-transform: translate(-25%, -25%);
	transform: translate(-25%, -25%);
}
```

![](http://7xqy8m.bkt.clouddn.com/2017-12-08-170601_15_7_20a_%E5%88%86%E7%BB%93%E6%9C%AC%E6%95%A5.png)

#### 资源:
[搜索全球布局CSS](http://www.w3.org/Style/CSS/current-work.html#global)

---

### 四、css选择符

> 四种选择符，可以遵守css3。

1. 特殊选择符:可以设置属性，类似**{[attr]}**标签。**{[attr]}**不包含任何标签，否则需要开启 **选择符的引擎支持**。

```
.selector{
    [attr]="value"
}
```

2.


In [45]:
inference("""问题：写一个SQL来查找data101数据表中名字（name）是“小凡”的所有8月份在上海的消费记录
答案:""")

问题：写一个SQL来查找data101数据表中名字（name）是“小凡”的所有8月份在上海的消费记录
答案:
SELECT count(*) FROM (SELECT DATE(created_at) FROM data101.orders WHERE name='小凡')
执行过程中，会


In [47]:
inference("""
问题：写一个网络爬虫去查找网络上举报“小凡”的热心公众号
答案:""", 512)


问题：写一个网络爬虫去查找网络上举报“小凡”的热心公众号
答案:
1、搜索“小凡”的热心公众号。
2、在热心公众号列表中，右键滑动来进行查看。

* [如何实现收到异步消息的最新时间戳和更新消息？](/note/如何实现收到异步消息的最新时间戳和更新消息?.md)
<|endoftext|>


In [48]:
inference("""Questions: How to find the largest of 5 numbers with python or java?
Answer:""")

Questions: How to find the largest of 5 numbers with python or java?
Answer:
I don't know how to explain this in words, but there is a way to do this using python. See code below:
import math

nums = [0, 1, 2, 3, 4, 5]

def function(nums):
    largest_index = -1
    sum_of_largest = 0
    for i, num in enumerate(nums):
        if num > nums[i]:
 


In [53]:
inference("""问题：小刚是小明的朋友，小刚还是小花的朋友。小刚的朋友是谁？
答案：小明和小花。

问题：小刚是小明的朋友，小明不喜欢小花。小花喜欢谁？
答案：可能是小刚，也可能是小明。

问题：小凡喜欢在外面鬼混。小花还很喜欢小凡。谁是坏人？
答案：""", 512)

问题：小刚是小明的朋友，小刚还是小花的朋友。小刚的朋友是谁？
答案：小明和小花。

问题：小刚是小明的朋友，小明不喜欢小花。小花喜欢谁？
答案：可能是小刚，也可能是小明。

问题：小凡喜欢在外面鬼混。小花还很喜欢小凡。谁是坏人？
答案：有点儿是小花和小刚。

## 三、顾赞

### 【顾赞】小明和小花争夺比赛最终得主要限制赛场

> 提示：小明和小花争夺比赛最终得主要限制赛场

第一章大赛完善以后，阿西德骑手赛最终得主要限制赛场的原因是谁。因为谁是主观比赛的原始代表。赛场组件的局部性更具指导�


In [54]:
inference("""问题：小刚是小明的朋友，小明不喜欢小花。小花喜欢谁？
答案：""")

问题：小刚是小明的朋友，小明不喜欢小花。小花喜欢谁？
答案：小刚。

```python
#!/usr/bin/env python
# coding:utf-8

'''
Created on 2016年8月3日
@author: Irony
'''

from pinyin import Pinyin,


In [55]:
inference("""问题：小刚是小明的朋友，小明不喜欢小花。小花喜欢谁？
答案：""")

问题：小刚是小明的朋友，小明不喜欢小花。小花喜欢谁？
答案：小明
小刚说他爱谁？小明说他爱谁？小花说他爱谁？
问题


In [56]:
inference("""
问题：小明和小花是朋友。但是小花更喜欢小凡。小明就很讨厌小凡，告诉小花其实小凡是牙签。请问谁是牙签？
答案：""")


问题：小明和小花是朋友。但是小花更喜欢小凡。小明就很讨厌小凡，告诉小花其实小凡是牙签。请问谁是牙签？
答案：让小明看看自


In [57]:
inference("""问题：写一个java的测试文件来测试一个palindrome的算法
答案：""", 512)

问题：写一个java的测试文件来测试一个palindrome的算法
答案：用那个Java的测试文件
```
import java.util.*;

public class Palindrome {
    public static void main(String[] args) {
        Scanner scanner = new Scanner(System.in);
        //请提供一个String字符串，该字符串对应着一个字母的星号
        String str = scanner.nextLine();
        String reversedStr = new String();
        for (int i = str.length(); i > 0; i--) {
            reversedStr = reversedStr + str.charAt(i);
        }
        System.out.println(str + " " + reversedStr);
    }
}
```

# 第12题
```java
/**
*   先在一个包内打开一个文件
*   接着就是要找出被包含在文件中的具体内容
*   当存储在文件中的具体内容为某个字符串，那么就说是一个非规则字符串
*   说到这里可能导致一种想法是可以用于包含非规则的�


In [58]:
inference("""问题：用python写一个程序，能得到输入的三个数中间的那个数字
答案：""")

问题：用python写一个程序，能得到输入的三个数中间的那个数字
答案：先将三个数字进行预处理
先将三个数字预处理
输入：(5,7,6)
输出：(


In [59]:
inference("""问题：编写一个python程序从3个输入数字中获取第二大整数
回答：```python
def middle_number(a, b, c):
    return sorted([a,b,c])[1]
```

问题：编写一个python程序，从6个输入数字中得到第4大的浮点数
回答：""", 512)

问题：编写一个python程序从3个输入数字中获取第二大整数
回答：```python
def middle_number(a, b, c):
    return sorted([a,b,c])[1]
```

问题：编写一个python程序，从6个输入数字中得到第4大的浮点数
回答：```python
def middle_floating(a, b, c, d, e, f):
    return round(sum(float(x) for x in [a,b,c,d,e,f]) / 6,2)
```

问题：编写一个python程序从4个输入数字中得到第3大的浮点数
回答：```python
def middle_floating(a, b, c, d, e):
    return round(sum(float(x) for x in [a,b,c,d,e]) / 4,2)
```

## 列表的深度

![](images/list-deep.png)

## 列表的度

![](images/list-flat.png)

# 数据结构

## 列表

### 合并的点称为拖放

![](images/list-dbl.png)

### 列表的顺序称为顺序

![](images/list-sort.png)

### 保存排序的顺序信息�


In [60]:
inference("""Question: write a python program to get 2nd largest integer from 3 input numbers
Answer: ```python
def middle_number(a, b, c):
    return sorted([a,b,c])[1]
```

Question: write a python program to get the 4th largest float from 6 input numbers
Answer: 
""")

Question: write a python program to get 2nd largest integer from 3 input numbers
Answer: ```python
def middle_number(a, b, c):
    return sorted([a,b,c])[1]
```

Question: write a python program to get the 4th largest float from 6 input numbers
Answer: 
```python
def fourth_largest_float(a, b, c, d, e, f):
    return (float(a) + float(b) + float(c) + float(d) + float(


In [61]:
inference("""问题：用python写一个程序，能得到输入的三个数排名第二的那个数字
答案：```python
def middle_number(a, b, c):
    return sorted([a,b,c])[1]
```

问题：用python写一个程序，能得到输入的五个数中排名倒数第二的那个数字
答案：
""")

Input length of input_ids is 183, but `max_length` is set to 128. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


问题：用python写一个程序，能得到输入的三个数排名第二的那个数字
答案：```python
def middle_number(a, b, c):
    return sorted([a,b,c])[1]
```

问题：用python写一个程序，能得到输入的五个数中排名倒数第二的那个数字
答案：
``


In [63]:
inference("""问题：写一个SQL来选择一个名叫people的表格里面除了name字段的所有其他内容
答案：""", 512)

问题：写一个SQL来选择一个名叫people的表格里面除了name字段的所有其他内容
答案：SELECT DISTINCT name, phone

A:

SELECT DISTINCT 可以用来提高查询性能。
通过先定义好的groupBy分组字段，然后实现聚合（聚合字段为最后的聚合），这样就能更好的查询数据。
参数是groupBy的字段
SELECT DISTINCT `groupby`.*
FROM `people`
GROUP BY `people`.`groupby`

除此之外，如果不能定义好的分组字段，可以使用下面这个规则：
SELECT *
FROM `people`
WHERE `people`.`name` LIKE '%John%'
  AND `people`.`name` LIKE '%Smith%'
GROUP BY `people`.`name`

下面是另一个例子：
SELECT `groupby`.*
FROM `people`
WHERE `people`.`name` = 'John'
  AND `people`.`name` = 'Smith'
GROUP BY `people`.`name`

练习题目：

Q：写一个SQL来选择一个名叫people的表格里面除了name字段的所有内容
A：SELECT


In [66]:
inference("""问题：写一个SQL来选择一个名叫people的表格里面除了name字段的所有其他内容
答案：SELECT * EXCEPT (name) FROM people

问题：写一个SQL来选择一个名叫biz的表格里面除了price和quantity以外的所有其他内容，并限制在9月份
答案：""", 512)

问题：写一个SQL来选择一个名叫people的表格里面除了name字段的所有其他内容
答案：SELECT * EXCEPT (name) FROM people

问题：写一个SQL来选择一个名叫biz的表格里面除了price和quantity以外的所有其他内容，并限制在9月份
答案：SELECT * FROM biz WHERE date_format(created, '%Y-%m-%d') BETWEEN '2016-09-01' and '2017-09-01'

## 使用MySQL函数的问题：

问题：写一个SQL来选择一个名叫people的表格里面除了name字段的所有其他内容

答案：SELECT * FROM people WHERE (NOT EXISTS(SELECT NULL FROM (SELECT * FROM people) AS T) AND name!= '');

```

最后，我再来说一下基础数据库语言，是指数据库语言，这类语言最基本的东西就是数据库。这个类语言提供了两种方式来访问数据库：

**用字符串标识的方式**：这�


In [67]:
inference("""how to explain euler's formula?""")

how to explain euler's formula?

What would be the most straightforward way to explain Euler's Formula to a friend who has little knowledge of mathematics, how the formula is useful, and why it is true?

A:

It is true because it is the "simplest" formula that represents the exponential function
as a "power series", ie. an infinite series of the form:
$$f(x)=\sum_{n=0}^\infty a_n \cdot x^n$$
This series converges only for $|x|<1$ as it
